# Tiny Autograd Tutorial - 스칼라 자동미분 구현하기

이 노트북에서는 PyTorch의 autograd와 유사한 자동미분(automatic differentiation) 엔진을 처음부터 구현해봅니다.

## 목표
- 연산 그래프(computation graph) 이해하기
- 역전파(backpropagation) 알고리즘 구현하기
- Chain rule을 통한 gradient 계산 이해하기

## 주요 개념

### 1. 자동미분(Automatic Differentiation)이란?
- 컴퓨터가 함수의 미분을 자동으로 계산하는 기법
- 딥러닝에서 gradient를 효율적으로 계산하는 핵심 기술

### 2. 연산 그래프(Computation Graph)
- 연산들의 순서와 의존관계를 나타내는 DAG(Directed Acyclic Graph)
- 각 노드는 값(Value)을 가지고, 엣지는 연산을 나타냄

### 3. Chain Rule
- 합성함수의 미분법칙: $\frac{dz}{dx} = \frac{dz}{dy} \cdot \frac{dy}{dx}$
- 역전파의 수학적 기초

## Step 1: Value 클래스 기본 구조

먼저 자동미분을 위한 기본 데이터 구조를 만들어봅시다.

In [2]:
from __future__ import annotations

import math
from dataclasses import dataclass, field
from typing import Callable


@dataclass
class Value:
    """스칼라 값과 그래디언트를 저장하는 클래스
    
    Attributes:
        data: 실제 스칼라 값
        grad: 이 노드에 대한 출력의 gradient
        _prev: 이 노드를 만든 입력 노드들의 집합
        _backward: 역전파 시 실행할 함수
    """
    data: float
    _prev: set[Value] = field(default_factory=set, repr=False)
    _backward: Callable[[], None] = field(default=lambda: None, repr=False)
    grad: float = 0.0

    def __post_init__(self) -> None:
        if not isinstance(self.data, (int, float)):
            raise TypeError("Value.data must be a number")
        self.data = float(self.data)

        def __repr__(self) -> str:
            return f"Value(data={self.data:.6f}, grad={self.grad:.6f})"
    
    def __hash__(self) -> int:
        return id(self)

    def __eq__(self, other) -> bool:
        return self is other

In [3]:
def append_item(x, items=[]):  # 가변 기본값
    items.append(x)
    return items

def append_item2(x, items=None):
    if items is None:
        items = []
    items.append(x)
    return items

print(append_item(1))  # [1]
print(append_item(2))  # [1, 2]  ← 이전 호출과 공유됨

print(append_item2(1))  # [1]
print(append_item2(2))  # [2]

[1]
[1, 2]
[1]
[2]


## Step 2: 덧셈 연산 구현

덧셈의 미분 규칙:
- $f(a, b) = a + b$
- $\frac{\partial f}{\partial a} = 1$
- $\frac{\partial f}{\partial b} = 1$

In [ ]:
def add_operation(self, other: float | Value) -> Value:
    """덧셈 연산 구현
    
    덧셈의 로컬 gradient는 항상 1입니다.
    out = self + other 일 때:
    - d(out)/d(self) = 1
    - d(out)/d(other) = 1
    """
    # other가 스칼라면 Value로 변환
    other = other if isinstance(other, Value) else Value(other)
    
    # 새로운 Value 노드 생성 (forward pass)
    out = Value(self.data + other.data, {self, other})

    def _backward() -> None:
        # Chain rule 적용: gradient 누적
        # out.grad는 상위 노드에서 전파된 gradient
        self.grad += out.grad * 1.0  # 덧셈의 로컬 gradient는 1
        other.grad += out.grad * 1.0

    out._backward = _backward
    return out

# Value 클래스에 메서드 추가
Value.__add__ = add_operation
Value.__radd__ = lambda self, other: self.__add__(other)

In [ ]:
hasattr(Value, "__add__")  # True가 나와야 정상

a = Value(3.0)

# 왼쪽이 숫자라서, int.__add__가 Value를 모르면 a.__radd__(2)가 호출됨
c1 = 2 + a   # __radd__ 경유 → __add__ 재사용
c2 = a + 2   # __add__

print(c1)
print(c2)

### 덧셈 테스트

In [ ]:
# 간단한 덧셈 예제
a = Value(2.0)
b = Value(3.0)
c = a + b

print(f"a = {a}")
print(f"b = {b}")
print(f"c = a + b = {c}")
print(f"c의 부모 노드들: {c._prev}")

## Step 3: 곱셈 연산 구현

곱셈의 미분 규칙 (Product Rule):
- $f(a, b) = a \times b$
- $\frac{\partial f}{\partial a} = b$
- $\frac{\partial f}{\partial b} = a$

In [ ]:
def mul_operation(self, other: float | Value) -> Value:
    """곱셈 연산 구현
    
    곱셈의 로컬 gradient는 상대방의 값입니다.
    out = self * other 일 때:
    - d(out)/d(self) = other.data
    - d(out)/d(other) = self.data
    """
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, {self, other})

    def _backward() -> None:
        # Product rule 적용
        print("backward is called, self.grad = ", self.grad, "other.grad = ", other.grad, "out.grad = ", out.grad)
        self.grad += other.data * out.grad  # d(a*b)/da = b
        other.grad += self.data * out.grad  # d(a*b)/db = a

    out._backward = _backward
    return out

Value.__mul__ = mul_operation
Value.__rmul__ = lambda self, other: self.__mul__(other)

### 곱셈 테스트

In [ ]:
# 곱셈 예제
x = Value(3.0)
y = Value(4.0)
z = x * y

print(f"x = {x}")
print(f"y = {y}")
print(f"z = x * y = {z}")

# 스칼라와의 곱셈도 가능
w = z * 2
print(f"w = z * 2 = {w}")

## Step 4: 활성화 함수 - tanh 구현

tanh 함수의 미분:
- $f(x) = \tanh(x)$
- $\frac{df}{dx} = 1 - \tanh^2(x)$

In [ ]:
def tanh_operation(self) -> Value:
    """tanh 활성화 함수 구현
    
    tanh의 미분: dtanh/dx = 1 - tanh(x)^2
    """
    t = math.tanh(self.data)
    out = Value(t, {self})

    def _backward() -> None:
        # tanh의 로컬 gradient
        self.grad += (1 - t**2) * out.grad

    out._backward = _backward
    return out

Value.tanh = tanh_operation

## Step 5: ReLU 활성화 함수 (선택사항)

ReLU 함수의 미분:
- $f(x) = \max(0, x)$
- $\frac{df}{dx} = \begin{cases} 1 & \text{if } x > 0 \\ 0 & \text{otherwise} \end{cases}$

In [ ]:
def relu_operation(self) -> Value:
    """ReLU 활성화 함수 구현
    
    ReLU의 미분: x > 0 일 때 1, 아니면 0
    """
    out = Value(self.data if self.data > 0 else 0.0, {self})

    def _backward() -> None:
        # ReLU의 로컬 gradient
        self.grad += (out.data > 0) * out.grad

    out._backward = _backward
    return out

Value.relu = relu_operation

## Step 6: 역전파(Backpropagation) 구현

역전파 알고리즘:
1. 연산 그래프를 위상정렬(topological sort)
2. 출력 노드의 gradient를 1로 설정
3. 역순으로 각 노드의 gradient 계산

In [ ]:
def backward_operation(self) -> None:
    """역전파 알고리즘 구현
    
    1. DFS로 그래프를 위상정렬
    2. 출력(self)의 gradient를 1로 설정
    3. 역순으로 각 노드의 _backward() 실행
    """
    # 위상정렬을 위한 리스트와 방문 집합
    topo: list[Value] = []
    visited: set[Value] = set()

    def build_topo(v: Value) -> None:
        """DFS로 위상정렬 구축"""
        if v not in visited:
            visited.add(v)
            # 자식 노드들을 먼저 방문
            for child in v._prev:
                build_topo(child)
            # 자식들 처리 후 현재 노드 추가
            topo.append(v)

    # 그래프 구축
    build_topo(self)
    
    # 출력의 gradient는 1
    self.grad = 1.0
    
    # 역순으로 gradient 전파
    for v in reversed(topo):
        v._backward()

Value.backward = backward_operation

## Step 7: 전체 구현 통합

이제 모든 구성요소를 하나의 클래스로 통합해봅시다.

In [ ]:
# 전체 Value 클래스 구현
from __future__ import annotations

import math
from dataclasses import dataclass, field
from typing import Callable, Set


@dataclass
class Value:
    data: float
    _prev: Set[Value] = field(default_factory=set, repr=False)
    _backward: Callable[[], None] = field(default=lambda: None, repr=False)
    grad: float = 0.0

    def __post_init__(self) -> None:
        if not isinstance(self.data, (int, float)):
            raise TypeError("Value.data must be a number")
        self.data = float(self.data)

    def __add__(self, other: float | Value) -> Value:
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, {self, other})

        def _backward() -> None:
            self.grad += out.grad
            other.grad += out.grad

        out._backward = _backward
        return out

    def __radd__(self, other: float | Value) -> Value:
        return self.__add__(other)

    def __mul__(self, other: float | Value) -> Value:
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, {self, other})

        def _backward() -> None:
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward
        return out

    def __rmul__(self, other: float | Value) -> Value:
        return self.__mul__(other)

    def tanh(self) -> Value:
        t = math.tanh(self.data)
        out = Value(t, {self})

        def _backward() -> None:
            self.grad += (1 - t**2) * out.grad

        out._backward = _backward
        return out

    def relu(self) -> Value:
        out = Value(self.data if self.data > 0 else 0.0, {self})

        def _backward() -> None:
            self.grad += (out.data > 0) * out.grad

        out._backward = _backward
        return out

    def backward(self) -> None:
        topo: list[Value] = []
        visited: set[Value] = set()

        def build(v: Value) -> None:
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build(child)
                topo.append(v)

        build(self)
        self.grad = 1.0
        for v in reversed(topo):
            v._backward()
    
    def __hash__(self) -> int:
        return id(self)

    def __eq__(self, other) -> bool:
        return self is other

    def __repr__(self) -> str:
        return f"Value(data={self.data:.6f}, grad={self.grad:.6f})"

## Step 8: 실제 예제 - 복잡한 함수의 gradient 계산

이제 구현한 자동미분 엔진을 테스트해봅시다.

테스트 함수: $f(a, b) = (a \times b + a) \times \tanh(b)$

In [ ]:
# 복잡한 함수 예제
def test_complex_function():
    # 입력값 정의
    a = Value(1.3)
    b = Value(-0.7)
    
    # Forward pass: f(a,b) = (a*b + a) * tanh(b)
    c = a * b  # a * b
    d = c + a  # a * b + a
    e = b.tanh()  # tanh(b)
    out = d * e  # (a * b + a) * tanh(b)
    
    print("=== Forward Pass ===")
    print(f"a = {a.data:.6f}")
    print(f"b = {b.data:.6f}")
    print(f"c = a * b = {c.data:.6f}")
    print(f"d = c + a = {d.data:.6f}")
    print(f"e = tanh(b) = {e.data:.6f}")
    print(f"out = d * e = {out.data:.6f}")
    
    # Backward pass
    out.backward()
    
    print("\n=== Backward Pass ===")
    print(f"∂out/∂a = {a.grad:.6f}")
    print(f"∂out/∂b = {b.grad:.6f}")
    
    return out, a.grad, b.grad

result = test_complex_function()
print(result)

## Step 9: 수치 미분과 비교 검증

우리의 자동미분이 올바른지 수치 미분(numerical differentiation)과 비교해봅시다.

In [ ]:
import numpy as np


def numerical_gradient(f, a0: float, b0: float, eps: float = 1e-6):
    """중앙 차분법을 사용한 수치 미분
    
    f'(x) ≈ [f(x+ε) - f(x-ε)] / 2ε
    """
    # a에 대한 편미분
    grad_a = (f(a0 + eps, b0) - f(a0 - eps, b0)) / (2 * eps)
    
    # b에 대한 편미분
    grad_b = (f(a0, b0 + eps) - f(a0, b0 - eps)) / (2 * eps)
    
    return grad_a, grad_b

def f_scalar(a0: float, b0: float) -> float:
    """테스트 함수의 스칼라 버전"""
    return (a0 * b0 + a0) * math.tanh(b0)

def verify_gradients():
    """자동미분과 수치미분 비교"""
    test_cases = [
        (1.3, -0.7),
        (0.5, 0.5),
        (-1.2, 2.0)
    ]
    
    for a0, b0 in test_cases:
        # 자동미분
        a, b = Value(a0), Value(b0)
        out = (a * b + a) * b.tanh()
        out.backward()
        
        # 수치미분
        grad_a_num, grad_b_num = numerical_gradient(f_scalar, a0, b0)
        
        # 상대 오차 계산
        def relative_error(x, y):
            denom = max(1.0, abs(x), abs(y))
            return abs(x - y) / denom
        
        err_a = relative_error(a.grad, grad_a_num)
        err_b = relative_error(b.grad, grad_b_num)
        
        print(f"\n테스트 케이스: a={a0}, b={b0}")
        print(f"  자동미분: ∂f/∂a = {a.grad:.8f}, ∂f/∂b = {b.grad:.8f}")
        print(f"  수치미분: ∂f/∂a = {grad_a_num:.8f}, ∂f/∂b = {grad_b_num:.8f}")
        print(f"  상대오차: a = {err_a:.2e}, b = {err_b:.2e}")
        print(f"  통과: {err_a < 1e-4 and err_b < 1e-4}")

verify_gradients()

## Step 10: 시각화 - 연산 그래프 그리기 (선택사항)

연산 그래프를 시각화해서 자동미분이 어떻게 동작하는지 더 잘 이해해봅시다.

In [ ]:
def trace_graph(root):
    """연산 그래프 추적"""
    nodes, edges = set(), set()
    nodes, edges = list(), set()
    
    def build(v):
        if v not in nodes:
            nodes.append(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    
    build(root)
    return nodes, edges

def draw_graph(root):
    """간단한 텍스트 기반 그래프 표현"""
    nodes, edges = trace_graph(root)
    
    print("\n=== 연산 그래프 구조 ===")
    print(f"노드 개수: {len(nodes)}")
    print(f"엣지 개수: {len(edges)}")
    
    print("\n노드 정보:")
    for i, node in enumerate(nodes):
        print(f"  Node {i}: data={node.data:.4f}, grad={node.grad:.4f}")
    
    print("\n연결 정보:")
    for src, dst in edges:
        print(f"  {src.data:.4f} -> {dst.data:.4f}")

# 예제 그래프 생성 및 시각화
a = Value(2.0)
b = Value(3.0)
c = a * b
d = c + a
e = d.tanh()

e.backward()
draw_graph(e)

## 연습 문제

이제 자동미분 엔진이 완성되었습니다! 다음 연습문제를 풀어보세요:

### 문제 1: 새로운 연산 추가하기
빼기(`__sub__`)와 나누기(`__truediv__`) 연산을 구현해보세요.

In [ ]:
# 연습: 빼기와 나누기 구현
def sub_operation(self, other):
    # TODO: 빼기 구현
    # 힌트: a - b = a + (-b)
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data - other.data, {self, other})

    def _backward() -> None:
        self.grad += out.grad * 1.0
        other.grad += out.grad * -1.0

    out._backward = _backward
    return out

def div_operation(self, other):
    # TODO: 나누기 구현
    # 힌트: a / b의 미분
    # d(a/b)/da = 1/b
    # d(a/b)/db = -a/b^2
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data / other.data, {self, other})

    def _backward() -> None:
        self.grad += out.grad * 1.0 / other.data
        other.grad += out.grad * -self.data / other.data**2

    out._backward = _backward
    return out


Value.__sub__ = sub_operation
Value.__rsub__ = lambda self, other: self.__sub__(other)
Value.__truediv__ = div_operation
Value.__rtruediv__ = lambda self, other: self.__truediv__(other)


x = Value(5.0) 
y = Value(2.0)
z = x - y
print(z)

z = x / y
print(z)

### 문제 2: 더 복잡한 함수 테스트
다음 함수의 gradient를 계산해보세요:
$f(x, y) = \frac{x^2 + y}{\tanh(x \cdot y)}$

In [ ]:
# 연습: 복잡한 함수의 gradient 계산
def complex_function_exercise():
    x = Value(1.0)
    y = Value(2.0)
    
    # TODO: f(x,y) = (x^2 + y) / tanh(x*y) 구현
    # 힌트: 거듭제곱은 x * x로 구현
    a = x * x
    b = a + y
    c = x * y
    out = b / c.tanh()
    
    out.backward()
    print(out, x.grad, y.grad)

complex_function_exercise()

## 정리

이 튜토리얼에서 우리는:

1. **연산 그래프**를 구축하는 `Value` 클래스를 만들었습니다
2. **Forward pass**에서 연산 결과와 함께 로컬 gradient를 저장했습니다
3. **Backward pass**에서 chain rule을 사용해 gradient를 계산했습니다
4. **위상정렬**을 사용해 올바른 순서로 gradient를 전파했습니다

### 핵심 개념 요약

- **자동미분**: 프로그램이 실행되면서 자동으로 미분값을 계산
- **Chain Rule**: 합성함수의 미분법칙, 딥러닝의 핵심
- **연산 그래프**: 계산 과정을 DAG로 표현
- **역전파**: 출력에서 입력으로 gradient를 전파

### 다음 단계

- 벡터/행렬 연산으로 확장 (Day 2)
- GPU 가속 지원
- 더 많은 연산과 최적화 기법 추가
- 실제 신경망 학습에 적용

이제 여러분은 PyTorch나 TensorFlow의 autograd가 어떻게 동작하는지 이해했습니다! 🎉

## 부록: 프로젝트 실행 방법

### 1. 의존성 설치
```bash
cd tiny_autograd_project
make setup  # 또는 pip install -r requirements.txt
```

### 2. 테스트 실행
```bash
make test  # pytest 실행
```

### 3. 스모크 테스트
```bash
make smoke  # 또는 python 50_eval/smoke.py
```

### 4. 코드 품질 검사
```bash
make fmt  # ruff, black, mypy 실행
```